In [1]:
import numpy as np
import pandas as pd

In [46]:
df = pd.read_csv('000157.KS.csv')
df.shape

(234, 7)

In [14]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
229,2019-01-29,75600.0,75600.0,74800.0,75500.0,75500.0,4
230,2019-01-30,75400.0,76900.0,75400.0,76900.0,76900.0,31
231,2019-01-31,75400.0,76700.0,75100.0,76500.0,76500.0,84
232,2019-02-01,76500.0,77000.0,75300.0,75900.0,75900.0,120
233,2019-02-21,74000.0,74700.0,72400.0,72400.0,72400.0,326


X :  오늘부터(오늘 포함) 과거 일주일 데이터  shape (m, 7일치, 6개 특징) 3d
Y :  내일의 Open                             shape (m, 1)               2d

1 2 3 4 5 6 7 8 9 10 일 
            7 (8)
  1 ~         8 (9)
    3  ~        9 (10)
(X, Y) 총 3세트를 만들수 있습니다
총세트 : 전체일수 - 7일 

In [47]:
sequence = 7
m = len(df) - sequence
len(df), m

(234, 227)

In [48]:
df.isna().sum().sum()

0

In [49]:
df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [50]:
vol = df.iloc[:,1]
vol.min(), vol.mean(), vol.std(), vol.max()

(70000.0, 75765.38461538461, 2912.463682386705, 83000.0)

In [51]:
df.shape

(234, 7)

In [52]:
x = np.zeros([m, sequence,df.shape[1]-1])
x.shape

(227, 7, 6)

In [57]:
arr = np.array(df.iloc[:, 1:])
arr_min = np.min(arr, axis=0, keepdims=True)# 축!! 0 
arr_max = np.max(arr, axis=0, keepdims=True)# 축!! 0 
arrn = (arr-arr_min) / (arr_max - arr_min)

In [59]:
for i in range(m):
    x[i] = arrn[i:i + sequence] #0 1 2 3 4 5 6
y = arrn[sequence:, 0]

In [60]:
x.shape, y.shape, np.min(y), np.max(y)

((227, 7, 6), (227,), 0.0, 1.0)

In [61]:
x_train = x[::2]
x_test = x[1::2]
y_train = y[::2]
y_test = y[1::2]

In [63]:
import tensorflow as tf
import tensorflow.contrib.layers as layers

In [66]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, sequence, 6])
Y = tf.placeholder(tf.float32, [None])
cell = tf.contrib.rnn.BasicLSTMCell(100)# 셀 갯수 RNN, LSTM, GRU
output, state = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
last_output = output[:, -1] # [m, 100]
h = layers.fully_connected(last_output, 1) # 몇번 더 
cost = tf.reduce_mean(tf.square(Y-h))
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for iter in range(100):
    _, co = sess.run([train, cost], {X:x_train, Y:y_train})
    if iter%10==0: print(iter, 'cost',co)

In [ ]:
predict = sess.run(h, {X:x_train})
plt.plot(y_train)
plt.plot(predict)